In [30]:
import pandas as pd

df_co2 = pd.read_csv("/content/co2-emissions-per-capita.csv")
df_co2.head()

,Entity,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,1949,0.001992
1,Afghanistan,1950,0.010837
2,Afghanistan,1951,0.011625
3,Afghanistan,1952,0.011468
4,Afghanistan,1953,0.013123


In [31]:
import pandas as pd
df_primary_energy = pd.read_csv("/content/primary-energy-cons.csv")
df_primary_energy.head()

,Entity,Code,Year,Primary energy consumption (TWh)
0,Afghanistan,AFG,1980,6.337187
1,Afghanistan,AFG,1981,7.289555
2,Afghanistan,AFG,1982,7.889244
3,Afghanistan,AFG,1983,9.882159
4,Afghanistan,AFG,1984,9.929878


Część 1: Plik CSV
Utwórz plik CSV zawierający połączone dane dla Polski:
- konsumpcja energii pierwotnej
- emisja CO2
- dane z poszczególnych lat

In [32]:
df_co2_poland = df_co2[df_co2['Entity'] == 'Poland'].copy()
df_primary_energy_poland = df_primary_energy[df_primary_energy['Entity'] == 'Poland'].copy()

display(df_co2_poland.head())
display(df_primary_energy_poland.head())


,Entity,Year,Annual CO₂ emissions (per capita)
19063,Poland,1800,0.045189
19064,Poland,1801,0.048900
19065,Poland,1802,0.049344
19066,Poland,1803,0.050177
19067,Poland,1804,0.049414


,Entity,Code,Year,Primary energy consumption (TWh)
9734,Poland,POL,1965,776.05330
9735,Poland,POL,1966,791.17820
9736,Poland,POL,1967,814.29987
9737,Poland,POL,1968,876.36676
9738,Poland,POL,1969,936.06604


In [33]:
df_merged_poland = pd.merge(df_co2_poland, df_primary_energy_poland, on='Year')
display(df_merged_poland.head())


,Entity_x,Year,Annual CO₂ emissions (per capita),Entity_y,Code,Primary energy consumption (TWh)
0,Poland,1965,7.893703,Poland,POL,776.05330
1,Poland,1966,7.996349,Poland,POL,791.17820
2,Poland,1967,8.138128,Poland,POL,814.29987
3,Poland,1968,8.597721,Poland,POL,876.36676
4,Poland,1969,9.076116,Poland,POL,936.06604


In [34]:
df_merged_poland.to_csv('poland_energy_co2.csv', index=False)

Stwórz plik Excel z osobnymi zakładkami dla każdego kraju:
- UE
- USA
- Chiny
- Indie

Każda zakładka powinna zawierać dane o konsumpcji energii i emisji CO2.

In [35]:
countries = ['European Union (27)', 'United States', 'China', 'India']

merged_dataframes = {}

for country in countries:
    df_co2_country = df_co2[df_co2['Entity'] == country].copy()
    df_primary_energy_country = df_primary_energy[df_primary_energy['Entity'] == country].copy()

    merged_df = pd.merge(df_co2_country, df_primary_energy_country, on='Year')
    merged_dataframes[country] = merged_df

# Create an Excel writer object
with pd.ExcelWriter('countries_energy_co2.xlsx') as writer:
    for country, df in merged_dataframes.items():
        df.to_excel(writer, sheet_name=country, index=False)

print("Excel file 'countries_energy_co2.xlsx' created successfully.")

Excel file 'countries_energy_co2.xlsx' created successfully.


Część 3: Baza danych SQLite
Utwórz bazę danych SQLite z tabelą zawierającą:
- kraj
- rok
- emisja CO2
- konsumpcja energii

In [36]:
df_merged = pd.merge(df_co2, df_primary_energy, on=['Year', 'Entity'], how='inner')
display(df_merged.head())

,Entity,Year,Annual CO₂ emissions (per capita),Code,Primary energy consumption (TWh)
0,Afghanistan,1980,0.133363,AFG,6.337187
1,Afghanistan,1981,0.165734,AFG,7.289555
2,Afghanistan,1982,0.190566,AFG,7.889244
3,Afghanistan,1983,0.230808,AFG,9.882159
4,Afghanistan,1984,0.252143,AFG,9.929878


In [37]:
df_co2 = pd.read_csv("/content/co2-emissions-per-capita.csv")
df_primary_energy = pd.read_csv("/content/primary-energy-cons.csv")

df_merged = pd.merge(df_co2, df_primary_energy, on=['Year', 'Entity'], how='inner')
display(df_merged.head())

,Entity,Year,Annual CO₂ emissions (per capita),Code,Primary energy consumption (TWh)
0,Afghanistan,1980,0.133363,AFG,6.337187
1,Afghanistan,1981,0.165734,AFG,7.289555
2,Afghanistan,1982,0.190566,AFG,7.889244
3,Afghanistan,1983,0.230808,AFG,9.882159
4,Afghanistan,1984,0.252143,AFG,9.929878


In [38]:
import sqlite3

conn = sqlite3.connect('energy_co2.db')

In [39]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE energy_data (
    country TEXT,
    year INTEGER,
    co2_emissions REAL,
    primary_energy REAL
)
''')

conn.commit()

OperationalError: table energy_data already exists

In [40]:
cursor = conn.cursor()

for index, row in df_merged.iterrows():
    cursor.execute('''
    INSERT INTO energy_data (country, year, co2_emissions, primary_energy)
    VALUES (?, ?, ?, ?)
    ''', (row['Entity'], row['Year'], row['Annual CO₂ emissions (per capita)'], row['Primary energy consumption (TWh)']))

conn.commit()

In [41]:
cursor.execute('SELECT * FROM energy_data LIMIT 5')
rows = cursor.fetchall()

for row in rows:
    print(row)

('Afghanistan', 1980, 0.13336322, 6.337187)
('Afghanistan', 1981, 0.16573393, 7.2895555)
('Afghanistan', 1982, 0.19056578, 7.8892436)
('Afghanistan', 1983, 0.2308076, 9.882159)
('Afghanistan', 1984, 0.25214335, 9.929878)


In [42]:
conn.close()